In [1]:
%cd ../../

/home/PJWSTK/s14028/Engine


In [2]:
import keras
import scipy.io as mat

from common import *
from augmentation import add_pmap
from augmentation import augmentation_data
from augmentation import augment_data

from CNN.CNN_v2_1 import CNN_v2_1

Using TensorFlow backend.


In [3]:
perspective = mat.loadmat("mall_dataset/perspective_roi.mat")["pMapN"]

perspective /= np.min(perspective)
perspective = np.round(perspective).astype(np.uint8)

train, test = data_sets()
image_tensors = train[0], test[0]
person_coo_tensors = train[1], test[1]
count_matrix = train[2], test[2]

image_train, image_test = image_tensors
person_coo_train, person_coo_test = person_coo_tensors
count_train, count_test = count_matrix
count_train = count_train.astype(np.uint16)
count_test = count_test.astype(np.uint16)

image_train = add_pmap(image_train, perspective)
image_test = add_pmap(image_test, perspective)

In [4]:
cnn_v2_1 = CNN_v2_1((480, 640, 4), split_into_parts=40)
cnn_v2_1.def_model()

In [5]:
images = cnn_v2_1._prepare_images(image_train)
anwsers = cnn_v2_1._prepare_anwsers(person_coo_train)

In [6]:
ones_count = (np.sum(anwsers == 1) * 0.25).astype(np.uint32)
zeros_count = (ones_count / 0.25 * 0.75).astype(np.uint32)
validation_length = (zeros_count + ones_count).astype(np.int32)

val_indices = np.concatenate([np.where(anwsers == 0)[0][:zeros_count],
                              np.where(anwsers == 1)[0][:ones_count]])

In [7]:
anwsers[val_indices[zeros_count:]] = 0

In [8]:
val_indices = -(images.shape[0] - val_indices)

In [9]:
generator = keras.preprocessing.image.ImageDataGenerator(rotation_range=20)

augmentation = augmentation_data(image_train, anwsers, 40)
augmented_data = augment_data(generator, augmentation, images, anwsers)
images, anwsers = augmented_data

In [10]:
anwsers[val_indices[zeros_count:]] = 1

images[-validation_length:], images[val_indices] = images[val_indices], images[-validation_length:]
anwsers[-validation_length:], anwsers[val_indices] = anwsers[val_indices], anwsers[-validation_length:]

In [11]:
parallel_model = keras.utils.multi_gpu_model(
        cnn_v2_1.model,
        gpus=2,
        cpu_merge=False)
parallel_model.compile(
        loss="binary_crossentropy", 
        optimizer="adam", 
        metrics=["accuracy"])

In [12]:
parallel_model.optimizer.lr.assign(1e-3)

<tf.Tensor 'Assign:0' shape=() dtype=float32_ref>

In [13]:
parallel_model.fit(images[:-validation_length], anwsers[:-validation_length], batch_size=20000, epochs=10,
                   validation_data=(images[-validation_length:], anwsers[-validation_length:]))

Train on 5311770 samples, validate on 51292 samples
Epoch 1/10
5311770/5311770 [==============================] - 132s 25us/step - loss: 74.6552 - acc: 0.5125 - val_loss: 2.9013 - val_acc: 0.7500
Epoch 2/10
5311770/5311770 [==============================] - 118s 22us/step - loss: 1.3929 - acc: 0.4935 - val_loss: 0.8473 - val_acc: 0.7500
Epoch 3/10
5311770/5311770 [==============================] - 118s 22us/step - loss: 0.8496 - acc: 0.4929 - val_loss: 0.7603 - val_acc: 0.7500
Epoch 4/10
5311770/5311770 [==============================] - 117s 22us/step - loss: 0.7892 - acc: 0.4929 - val_loss: 0.7342 - val_acc: 0.7500
Epoch 5/10
5311770/5311770 [==============================] - 117s 22us/step - loss: 0.7697 - acc: 0.4929 - val_loss: 0.7193 - val_acc: 0.7500
Epoch 6/10
5311770/5311770 [==============================] - 120s 23us/step - loss: 0.7594 - acc: 0.4929 - val_loss: 0.7126 - val_acc: 0.7500
Epoch 7/10
5311770/5311770 [==============================] - 125s 24us/step - loss: 0.75

In [14]:
parallel_model.optimizer.lr.assign(1e-1)

<tf.Tensor 'Assign_1:0' shape=() dtype=float32_ref>

In [15]:
parallel_model.fit(images[:-validation_length], anwsers[:-validation_length], batch_size=20000, epochs=10,
                   validation_data=(images[-validation_length:], anwsers[-validation_length:]))

Train on 5311770 samples, validate on 51292 samples
Epoch 1/10
5311770/5311770 [==============================] - 123s 23us/step - loss: 0.7451 - acc: 0.4929 - val_loss: 0.7013 - val_acc: 0.7500
Epoch 2/10
5311770/5311770 [==============================] - 123s 23us/step - loss: 0.7434 - acc: 0.4929 - val_loss: 0.6998 - val_acc: 0.7500
Epoch 3/10
5311770/5311770 [==============================] - 122s 23us/step - loss: 0.7428 - acc: 0.4929 - val_loss: 0.6990 - val_acc: 0.7500
Epoch 4/10
5311770/5311770 [==============================] - 124s 23us/step - loss: 0.7423 - acc: 0.4929 - val_loss: 0.6986 - val_acc: 0.7500
Epoch 5/10
5311770/5311770 [==============================] - 124s 23us/step - loss: 0.7419 - acc: 0.4929 - val_loss: 0.6997 - val_acc: 0.7500
Epoch 6/10
5311770/5311770 [==============================] - 125s 24us/step - loss: 0.7417 - acc: 0.4929 - val_loss: 0.6974 - val_acc: 0.7500
Epoch 7/10
5311770/5311770 [==============================] - 126s 24us/step - loss: 0.741

In [16]:
parallel_model.optimizer.lr.assign(1e-4)

<tf.Tensor 'Assign_2:0' shape=() dtype=float32_ref>

In [17]:
parallel_model.fit(images[:-validation_length], anwsers[:-validation_length], batch_size=20000, epochs=10,
                   validation_data=(images[-validation_length:], anwsers[-validation_length:]))

Train on 5311770 samples, validate on 51292 samples
Epoch 1/10
5311770/5311770 [==============================] - 125s 23us/step - loss: 0.7409 - acc: 0.4929 - val_loss: 0.6977 - val_acc: 0.7500
Epoch 2/10
5311770/5311770 [==============================] - 125s 24us/step - loss: 0.7409 - acc: 0.4929 - val_loss: 0.6960 - val_acc: 0.7500
Epoch 3/10
5311770/5311770 [==============================] - 124s 23us/step - loss: 0.7408 - acc: 0.4929 - val_loss: 0.6957 - val_acc: 0.7500
Epoch 4/10
5311770/5311770 [==============================] - 124s 23us/step - loss: 0.7408 - acc: 0.4929 - val_loss: 0.6983 - val_acc: 0.7500
Epoch 5/10
5311770/5311770 [==============================] - 124s 23us/step - loss: 0.7407 - acc: 0.4929 - val_loss: 0.6983 - val_acc: 0.7500
Epoch 6/10
5311770/5311770 [==============================] - 124s 23us/step - loss: 0.7406 - acc: 0.4929 - val_loss: 0.6972 - val_acc: 0.7500
Epoch 7/10
5311770/5311770 [==============================] - 124s 23us/step - loss: 0.740

In [19]:
parallel_model.save_weights("CNN/CNN_v2/weights/cnn_v2_1_40_1e_4_10")

In [ ]:
parallel_model.load_weights("CNN/CNN_v2/weights/cnn_v2_1_40_1e_4_10")